# Step 1: Load Data

In [0]:
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split

dataset_path = 'data/fer2013.csv'
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
width, height = 48, 48

def load_data():
    data = pd.read_csv(dataset_path)
    pixels = data['pixels'].tolist()
    faces = []
    for pixel_sequence in pixels:
        pixels_1D = pixel_sequence.split(' ')
        pixels_2D = np.asarray(pixels_1D).reshape(width, height)
        face = np.array(pixels_2D, 'float32')  #shape = (48,48)
        faces.append(face)                     #shape = (35887, 48, 48)
    faces = np.expand_dims(faces, -1)          #shape = (35887, 48, 48, 1)  grayscale is 1 only; while RGB is 3
    faces = faces / 255                        # Normalized
    emotions = keras.utils.to_categorical(data['emotion'], num_classes) #one hot encode 7 categories
    return faces, emotions

faces, emotions = load_data()

Using TensorFlow backend.


### Remarks

pixels_1D
[70.,80.,82.,72.,58.,...]

pixels_2D
[array([[ 70.,  80.,  82., ...,  52.,  43.,  41.],
        [ 65.,  61.,  58., ...,  56.,  52.,  44.],
        [ 50.,  43.,  54., ...,  49.,  56.,  47.],
        ...,
        [ 91.,  65.,  42., ...,  72.,  56.,  43.],
        [ 77.,  82.,  79., ..., 105.,  70.,  46.],
        [ 77.,  72.,  84., ..., 106., 109.,  82.]], dtype=float32)]
        
Normalized  
array([[0.27450982, 0.3137255 , 0.32156864, ..., 0.20392157, 0.16862746,
        0.16078432],
       [0.25490198, 0.23921569, 0.22745098, ..., 0.21960784, 0.20392157,
        0.17254902],
       [0.19607843, 0.16862746, 0.21176471, ..., 0.19215687, 0.21960784,
        0.18431373],
       ...,
       [0.35686275, 0.25490198, 0.16470589, ..., 0.28235295, 0.21960784,
        0.16862746],
       [0.3019608 , 0.32156864, 0.30980393, ..., 0.4117647 , 0.27450982,
        0.18039216],
       [0.3019608 , 0.28235295, 0.32941177, ..., 0.41568628, 0.42745098,
        0.32156864]], dtype=float32)
        
Emotions
array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

# Step 2: Define Model

In [0]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2
import pandas as pd
import cv2
import numpy as np
 
# parameters
batch_size = 32
num_epochs = 8
input_shape = (48, 48, 1)
verbose = 1
num_classes = 7
patience = 50
base_path = 'models/'
l2_regularization=0.01
 

# model parameters
regularization = l2(l2_regularization)
 
# base
img_input = Input(input_shape)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
 
# module 1
residual = Conv2D(16, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 2
residual = Conv2D(32, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 3
residual = Conv2D(64, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 4
residual = Conv2D(128, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
x = Conv2D(num_classes, (3, 3), padding='same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax',name='predictions')(x)

Instructions for updating:
Colocations handled automatically by placer.


# Step 3: Generator and Train/Test Splitting

In [0]:
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

# Step 4: Cross Validation Model Training

In [0]:
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5)
kf.get_n_splits(faces)

index = 1;
evaluation_result_accuracy = []
evaluation_result_precision = []
evaluation_result_recall = []
evaluation_result_fscore = []
evaluation_result_support = []


for train_index, test_index in kf.split(faces):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = faces[train_index], faces[test_index]
    
    y_train, y_test = emotions[train_index], emotions[test_index]
    # callbacks
    log_file_path = base_path + '_emotion_training' + str(index) + '.log'
    csv_logger = CSVLogger(log_file_path, append=False)
    early_stop = EarlyStopping('val_loss', patience=patience)
    reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
    trained_models_path = base_path + 'model_' + str(index)
    model_names = trained_models_path + '.hdf5'
    model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
    callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]
    train_generator = data_generator.flow(x_train, y_train, batch_size=batch_size)
    
    # base
    img_input = Input(input_shape)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # module 1
    residual = Conv2D(16, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)
    x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 2
    residual = Conv2D(32, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)
    x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 3
    residual = Conv2D(64, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)
    x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 4
    residual = Conv2D(128, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)
    x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])
    x = Conv2D(num_classes, (3, 3), padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax',name='predictions')(x)
    
    model = Model(img_input, output)
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    model.summary()
    model.fit_generator(
                train_generator, 
                steps_per_epoch=len(x_train) / batch_size, 
                epochs=num_epochs, verbose=1, callbacks=callbacks,
                validation_data=(x_test,y_test)) #train for randomly selected one
    index += 1;
    y_pred_prob=model.predict(x_test)
    y_pred = []
    for i in range(y_pred_prob.shape[0]):
        y_result = [0] * 7
        y_result[np.argmax(y_pred_prob[i])] = 1
        y_pred.append(y_result)
    y_pred = np.array(y_pred)
    evaluation_result = precision_recall_fscore_support(y_test, y_pred)
    evaluation_result_accuracy.append(accuracy_score(y_test, y_pred))
    evaluation_result_precision.append(evaluation_result[0])
    evaluation_result_recall.append(evaluation_result[1])
    evaluation_result_fscore.append(evaluation_result[2])
    evaluation_result_support.append(evaluation_result[3])


TRAIN: [ 7178  7179  7180 ... 35884 35885 35886] TEST: [   0    1    2 ... 7175 7176 7177]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 46, 46, 8)    72          input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 46, 46, 8)    32          conv2d_8[0][0]                   
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 46, 46, 8)    0           batch_normalization_15[0][0]     
__________________

Epoch 1/8
 53/897 [>.............................] - ETA: 3:41 - loss: 2.0498 - acc: 0.2146

In [0]:
for i in range(len(evaluation_result_accuracy)):
    print('Accuracy of ', i, ' is: ', evaluation_result_accuracy[i], '\n',
          'precision, recall, F-score and support for model  of ', i, ' are', '\n',
          evaluation_result_precision[i], '\n',
          evaluation_result_recall[i], '\n',
          evaluation_result_fscore[i], '\n',
          evaluation_result_support[i], '\n'
         )
print('The average accuracy for models is: ', 
          np.mean(evaluation_result_accuracy), '\n',
          'precision, recall, F-score and support', '\n',
          np.mean(evaluation_result_precision, axis=0), '\n',
          np.mean(evaluation_result_recall, axis=0), '\n',
          np.mean(evaluation_result_fscore, axis=0), '\n',
          np.mean(evaluation_result_support, axis=0)
         )

Accuracy of  0  is:  0.5341320702145445 
 precision, recall, F-score and support for model  of  0  are 
 [0.36393805 0.65       0.50657895 0.8013544  0.52072072 0.57926829
 0.42766296] 
 [0.64195122 0.12380952 0.07354346 0.78064871 0.2410342  0.74705111
 0.66147541] 
 [0.46452524 0.208      0.12844037 0.79086605 0.3295325  0.65254722
 0.51947216] 
 [1025  105 1047 1819 1199  763 1220] 

Accuracy of  1  is:  0.5583728057954862 
 precision, recall, F-score and support for model  of  1  are 
 [0.34821877 0.50793651 0.5401662  0.81095101 0.50821596 0.71370968
 0.5006993 ] 
 [0.71399177 0.26446281 0.18913676 0.80491991 0.35032362 0.69140625
 0.5499232 ] 
 [0.46812816 0.34782609 0.28017241 0.8079242  0.41475096 0.70238095
 0.52415813] 
 [ 972  121 1031 1748 1236  768 1302] 

Accuracy of  2  is:  0.6022014769402257 
 precision, recall, F-score and support for model  of  2  are 
 [0.53846154 0.31527094 0.45771144 0.78522838 0.52794411 0.80969479
 0.4897343 ] 
 [0.47727273 0.54237288 0.35148042

In [0]:
print(evaluation_result_accuracy)

[0.5341320702145445, 0.5583728057954862, 0.6022014769402257, 0.6194788909014909, 0.6265849240629789]


# Step 5: Model Training

In [0]:
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support

num_epochs = 100
trained_models_path = base_path + 'model_Full_Train'
model_names = trained_models_path + '.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]
train_generator = data_generator.flow(faces, emotions, batch_size=batch_size)
model.fit_generator(
                train_generator, 
                steps_per_epoch=len(x_train) / batch_size, 
                epochs=num_epochs, verbose=1, callbacks=callbacks,
                validation_data=(x_test,y_test)) #train for randomly selected one

Epoch 1/100
898/897 [==============================] - 175s 195ms/step - loss: 0.9844 - acc: 0.6346 - val_loss: 0.9957 - val_acc: 0.6323

Epoch 00001: val_loss improved from inf to 0.99567, saving model to models/model_Full_Train.hdf5
Epoch 2/100
898/897 [==============================] - 177s 197ms/step - loss: 0.9927 - acc: 0.6317 - val_loss: 1.0084 - val_acc: 0.6302

Epoch 00002: val_loss did not improve from 0.99567
Epoch 3/100
898/897 [==============================] - 182s 202ms/step - loss: 0.9829 - acc: 0.6313 - val_loss: 0.9794 - val_acc: 0.6359ss: 0.9825 - acc: 0.

Epoch 00003: val_loss improved from 0.99567 to 0.97936, saving model to models/model_Full_Train.hdf5
Epoch 4/100
898/897 [==============================] - 182s 203ms/step - loss: 0.9796 - acc: 0.6335 - val_loss: 0.9635 - val_acc: 0.6427

Epoch 00004: val_loss improved from 0.97936 to 0.96346, saving model to models/model_Full_Train.hdf5
Epoch 5/100
898/897 [==============================] - 187s 208ms/step - loss:

898/897 [==============================] - 278s 309ms/step - loss: 0.8132 - acc: 0.6987 - val_loss: 0.7858 - val_acc: 0.7105

Epoch 00081: val_loss did not improve from 0.78040
Epoch 82/100
898/897 [==============================] - 275s 306ms/step - loss: 0.8144 - acc: 0.7014 - val_loss: 0.7843 - val_acc: 0.7142

Epoch 00082: val_loss did not improve from 0.78040
Epoch 83/100
898/897 [==============================] - 274s 305ms/step - loss: 0.8037 - acc: 0.7018 - val_loss: 0.7833 - val_acc: 0.7114

Epoch 00083: val_loss did not improve from 0.78040
Epoch 84/100
898/897 [==============================] - 277s 309ms/step - loss: 0.8055 - acc: 0.7032 - val_loss: 0.7829 - val_acc: 0.7095

Epoch 00084: val_loss did not improve from 0.78040
Epoch 85/100
898/897 [==============================] - 279s 310ms/step - loss: 0.8127 - acc: 0.6977 - val_loss: 0.7738 - val_acc: 0.7139

Epoch 00085: val_loss improved from 0.78040 to 0.77379, saving model to models/model_Full_Train.hdf5
Epoch 86/100
